In [1]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import math
import csv

In [2]:
with open('edge.csv', newline = '') as f:
    reader = csv.reader(f)
    wg = list(reader)
wg = wg[1:]
wg

[['1', '2', '5'],
 ['2', '3', '4'],
 ['2', '4', '1'],
 ['3', '4', '5'],
 ['3', '5', '5'],
 ['4', '7', '3'],
 ['4', '8', '5'],
 ['5', '6', '1'],
 ['7', '9', '5']]

In [26]:
df = pd.DataFrame(wg)
df = df.astype({2: 'int'})

weight = df[2].max()
weight = str(weight)
weight

'5'

In [27]:
supernode = []
def IsSupernode(n):
    for i in range(0, len(supernode)):
        for j in range(0, len(supernode[i])):
            if n == supernode[i][j]:
                return i
    return -1

In [28]:
superedge = []
idx = 0

for i in range(0, len(wg)):
    if wg[i][2] == weight:
        if IsSupernode(wg[i][0]) >= 0: # source node가 supernode에 존재한다면
            row = IsSupernode(wg[i][0])
            supernode[row].append(wg[i][1])
        else:
            supernode.append([])
            supernode[len(supernode)-1].append(wg[i][0])
            supernode[len(supernode)-1].append(wg[i][1])
    else:
        superedge.append([])
        superedge[idx].append(wg[i][0])
        superedge[idx].append(wg[i][1])
        superedge[idx].append(wg[i][2])
        idx += 1
            
print(supernode)
print(superedge)

[['1', '2'], ['3', '4', '5', '8'], ['7', '9']]
[['2', '3', '4'], ['2', '4', '1'], ['4', '7', '3'], ['5', '6', '1']]


In [29]:
for i in range(0, len(superedge)):
    for j in range(0, 2):
        if IsSupernode(superedge[i][j]) >= 0:
            row = IsSupernode(superedge[i][j])
            superedge[i][j] = supernode[row][0]

superedge

[['1', '3', '4'], ['1', '3', '1'], ['3', '7', '3'], ['3', '6', '1']]

In [30]:
output = []

def IsSuperedge(s, t):
    for i in range(0, len(output)):
        if (s == output[i][0]) and (t == output[i][1]):
            return i
    return -1

In [31]:
idx2 = 0

for i in range(0, len(superedge)):
    s = superedge[i][0]
    t = superedge[i][1]
    w = superedge[i][2]
    if IsSuperedge(s, t) >= 0:
        # 있을 경우
        row = IsSuperedge(s, t)
        output[row][2] = float(output[row][2]) + float(w)
        output[row][3] += 1
    else:
        output.append([])
        output[idx2].append(s)
        output[idx2].append(t)
        output[idx2].append(w)
        output[idx2].append(1)
        idx2 += 1

output

[['1', '3', 5.0, 2], ['3', '7', '3', 1], ['3', '6', '1', 1]]

In [32]:
for i in range(0, len(output)):
    if output[i][3] > 1:
        output[i][2] /= output[i][3]

output

[['1', '3', 2.5, 2], ['3', '7', '3', 1], ['3', '6', '1', 1]]

In [33]:
output = pd.DataFrame(output)
output

,0,1,2,3
0,1,3,2.5,2
1,3,7,3,1
2,3,6,1,1


In [34]:
output.columns = ['Source', 'Target', 'Weight', 'Count']
output

,Source,Target,Weight,Count
0,1,3,2.5,2
1,3,7,3,1
2,3,6,1,1


In [35]:
output = output.drop(['Count'], axis = 1)
output

,Source,Target,Weight
0,1,3,2.5
1,3,7,3
2,3,6,1


In [36]:
output.to_csv("edge.csv", index = False)